In [22]:
import pandas as pd
import numpy as np
import os
import datetime



# d = date_data[0]
# print(type(d))
# d.month
# d.date()
# d.date() + datetime.timedelta(days=2)

In [73]:
# construct dataset
# prompting


class GeneratePrompt:
    def __init__(self, type) -> None:
        self.type = type
        self.template = ["我想在[YY年MM月DD日]买一注[NN]彩票，哪个号码中奖概率最高？",
                    "我计划在[YY年MM月DD日]购买一注[NN]彩票，哪些号码中奖的几率最大？",
                    "在[YY年MM月DD日]，我想买一注[NN]彩票，应该选择哪些号码？",
                    "如果我在[YY年MM月DD日]买一注[NN]彩票，哪些号码最有可能中奖？",
                    "请告诉我在[YY年MM月DD日]购买[NN]彩票时，哪些号码中奖概率最高？",
                    "我打算在[YY年MM月DD日]买一注[NN]彩票，哪些号码最有可能中奖？", 
                    "在[YY年MM月DD日]，我想买一注[NN]彩票，中奖概率最高的号码是什么？",
                    "如果我在[YY年MM月DD日]购买一注[NN]彩票，哪些号码最有希望中奖？",
                    "请预测在[YY年MM月DD日]购买[NN]彩票时，哪些号码中奖的几率最大？",
                    "预测一下[YY年MM月DD日]的[NN]彩票开奖号码。",
                    "请预测[YY年MM月DD日]的[NN]彩票中奖号码。",
                    "你能告诉我[YY年MM月DD日]的[NN]彩票可能的中奖号码吗？",
                    "预测一下在[YY年MM月DD日]，[NN]彩票的中奖号码会是什么？",
                    "请预测[YY年MM月DD日]的[NN]彩票号码。",
                    "你能预测[YY年MM月DD日]的[NN]彩票中奖号码吗？",
                    "请告诉我[YY年MM月DD日]的[NN]彩票可能的中奖号码。",
                    "预测一下[YY年MM月DD日]，[NN]彩票的中奖号码。",
                    "你能预测一下[YY年MM月DD日]的[NN]彩票号码吗？"]
        
        self.idx_number = len(self.template)
        
    def input_prompt(self, date: pd.Timestamp):
        prompt_idx = np.random.randint(self.idx_number)
        prompt = self.template[prompt_idx]
        date_text = f"{date.year}年{date.month}月{date.day}日"
        new_input = prompt.replace("[YY年MM月DD日]", date_text)
        new_input = new_input.replace("[NN]",self.type)
        return new_input
    
    def output_prompt(self, red_balls, blue_balls):
        red_balls_str = f"{red_balls[0]}, {red_balls[1]}, {red_balls[2]}, {red_balls[3]}, {red_balls[4]}, {red_balls[5]}"
        new_output = f"我的预测是：红球为{red_balls_str}，蓝球为{blue_balls}"
        return new_output

# gp = GeneratePrompt()

# input_1 = gp.input_prompt(date_data[2])
# print(input_1)

# output_1 = gp.output_prompt(red_balls[2],blue_balls[2])
# print(output_1)
# date_data = file["Date"]
# date_data = np.datetime64(date_data)
# date1 = date_data[0]
# year1 = date1.year()

class SSQDataLoader:
    def __init__(self, path) -> None:
        self.prompter = GeneratePrompt(type="双色球")
        file = pd.read_csv(path,delimiter=" ")
        file.columns = ['ID', 'Date', 'Red1', 'Red2', 'Red3', 'Red4', 'Red5',
                'Red6', 'Blue', 'Out1', 'Out2', 'Out3', 'Out4', 'Out5',
                  'Out6', 'Num13', 'Num14', 'Num15', 'Num16', 'Num17',
                    'Num18', 'Num19', 'Num20', 'Num21', 'Num22', 'Num23',
                      'Num24', 'Num25', 'Num26']
        self.date_data_interp = []
        self.issue_data_interp = []
        self.red_balls_data_interp = []
        self.blue_balls_data_interp = []
        self._interpolate(file)
        self.enquire_list, self.answer_list = self._prompt()

        

    def _interpolate(self,file) -> None:

        date_data = pd.to_datetime(file["Date"])
        # interpolate with date
        red_balls = np.column_stack([np.array(file["Out1"]), 
                    np.array(file["Out2"]),
                    np.array(file["Out3"]),
                    np.array(file["Out4"]),
                    np.array(file["Out5"]),
                    np.array(file["Out6"])])
        blue_balls = np.array(file["Blue"])

        for idx in range(len(date_data)-1):
            date_now = date_data[idx]
            date_next = date_data[idx+1]
            issue_next = file["ID"][idx+1]
            red_balls_next = red_balls[idx+1]
            blue_balls_next = blue_balls[idx+1]
            delta_date = date_next - date_now
            # print(type(delta_date))
            delta_date_days = delta_date.days
            # print(int(delta_date_days))
            interpolated_date = [date_now + pd.Timedelta(days=day) for day in range(delta_date_days) ]
            interpolated_issue = [issue_next for day in range(delta_date_days)]
            interpolated_red_ball = [red_balls_next for day in range(delta_date_days)]
            interpolated_blue_ball = [blue_balls_next for day in range(delta_date_days)]
            self.date_data_interp.extend(interpolated_date)
            self.issue_data_interp.extend(interpolated_issue)
            self.red_balls_data_interp.extend(interpolated_red_ball)
            self.blue_balls_data_interp.extend(interpolated_blue_ball)

    def _prompt(self):
        enquire_list = []
        answer_list = []
        for idx,_ in enumerate(self.date_data_interp):
            enquire = self.prompter.input_prompt(self.date_data_interp[idx])
            answer = self.prompter.output_prompt(self.red_balls_data_interp[idx],self.blue_balls_data_interp[idx])
            enquire_list.append(enquire)
            answer_list.append(answer)    

        return enquire_list, answer_list
    
    def __getitem__(self, index):
        return self.enquire_list[index], self.answer_list[index]
    
    def __len__(self):
        return len(self.answer_list)

在2003年3月6日，我想买一注双色球彩票，应该选择哪些号码？
我的预测是：红球为13, 4, 7, 25, 6, 10，蓝球为3


In [ ]:
file_name = "./data/ssq_asc.txt"
my_dataset = SSQDataLoader(file_name)

import mindspore.dataset as ds
dataset = ds.GeneratorDataset(my_dataset, ["enquire", "answer"], shuffle=True)